In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Prompt Engineering

With the recent success of Assistants (like ChatGPT)
- it is easy to reach the mistaken conclusion
- that the Assistant is "reasoning"
- when, in fact, it is doing nothing more than text-completion


## Review: Auto-regressive behavior of an LLM

The user
- inputs a sequence $\x$ (e.g., the "prompt", i.e., a request for generating a response)
- and expects a sequence $\y$ (the response)
- generated by the LLM according to probability distribution
$$
p( \y | \x )
$$

The response sequence $\y$ is generated *auto-regressively*:

At position $\tt$ of the output,
- the Large Language Model  predicts the next token.
$$
\hat{\x}_\tp \in \pr{ \x_\tp | \x_{[0..\tt-1]} }
$$
- conditional on all preceding tokens $\x_{[0..\tt-1]}$ in the sequence $\x$
    - the conditioning input is called the *context*
- and extends the context by appending the prediction 
$$
\x = \x_{[0..\tt-1]} + \hat\x_\tp
$$

Thus, at any step,  $\x$ consists of
-  the original user prompt as a prefix
- followed by a suffix of the partially generated response

## Prompt Engineering: maximizing the chances of achieving a good response

Given the auto-regressive generation process
- the final response $\y$ is therefore
- *conditioned on all previously generated response tokens*
- it is *path dependent*

In order to generate a "high quality" $\y$, we have to be aware of the path.



*Prompt engineering*
- is a collection of techniques
- that attempt to generate paths
- that lead to better responses

For example:
- many tasks involve multiple steps of reason
- asking
    - directly for the answer
    - is less likely to produce a correct response
    - than asking for a step-by-step explanation to *proceed* the answer
- that is
    - having the LLM's response include the individual steps before the answer
    - conditions it to produce the correct answer
        - just like a human !

To illustrate: the following prompt involves a task whose solution has multiple steps of arithmetic reasoning

            Each can contains 3 balls.
            I start with 5 cans
            At the end, all cans are empty except for one can with 2 balls.

            How many balls did I use ?

It is not reasonable to expect text-completion to *immediately* generate the correct response.

We can improve the chances of the LLM generating a correct response
- just by appending 

        Let's think step by step
    
to the prompt !

Here is ChatGPT's response:

<img src="images/cot_prompt_step_by_step.png">

Why does adding a simple request to think step-by-step work ?

- The request causes the model to generate the response as a sequence of small steps
- Step $\tt$ is conditioned on all steps $\tt' \lt \tt$
- The probability of a correct answer on a small step is higher than on a large step (i.e, straight to answer)

The step-by-step approach
- simulates reasoning
- by turning it into text completion

## A word on Assistants

The LLM's with which you may be familiar (e.g. ChatGPT) have been fine-tuned in several ways
- to be a helpful assistant
    - presume that your prompt is a request for service, a question that needs and answer, etc.
- to be conversational
- to be harmless and not dangerous

Hence, your experience with an "LLM" may not correspond to our description of an LLM that has not been fine-tuned.

It is important to remember
- that an LLM has **no memory**
- The output $\hat\y_\tp$ is solely a function of the prompt $\x_{[0..\tt-1]}$
- So how is it that the Assistant seems to "remember" the prior parts of the interaction ?

An interaction with an Assistant is often a multi-round conversation
- in round $i \ge 0$
    -  you enter prompt $\x^\ip$; get response $\hat\y^\ip$
- the *context* used to condition the response $\hat\y^{(i+1)}$
    - is prompt $\x^{(i+1)}$
    - concatenated with the prompt/responses of earlier rounds
- So 
$$
\hat\y^{(i+1)} \in \pr{ \y |  \x^{(i+1)}, \hat\y^\ip, \x^\ip, \ldots, \x^{(0)}, \hat\y^{(0)} }
$$

The Assistant "remembers" the entire conversation only by having it be part of the prompt.

# Resources

There a lots of "guides" (some paid) that purport to turn you into a Prompting Wizard.

Many of these are anecdotal.  We prefer measurement
- guides that summarize empirical studies
- and provide reference to the source paper

## [LearnPrompting.org Course](https://learnprompting.org/docs/intro)

A fairly simple (free and open source) course
- great way to find out what is interesting
- **and** has references to papers so as to enable deeper understanding

For example, some [basics](https://learnprompting.org/docs/category/-basics)
- [role playing](https://learnprompting.org/docs/basics/roles) to control the style of output
- [giving instructions](https://learnprompting.org/docs/basics/instructions) to *precisely* define the task

## [Prompting Guide](https://www.promptingguide.ai/)

Another fairly simple guide (ignore the promoted -- and paid -- course)
- also has references to papers

# Case study


- https://www.promptingguide.ai/applications/workplace_casestudy

You can delve into various prompting techniques by examining the resources.

As a short-cut
- we will describe a few of the techniques
- as part of a study evaluating techniques

One team decided to [measure the performance](https://arxiv.org/pdf/2303.07142.pdf) of various prompting techniques
- using a **single** task as a case study
- may not be able to generalize to other tasks

Regardless of the limitations, a comparison is valuable.

## Methodology

The task is a binary Classification task
- given a job posting
- classify whether the job listed is appropriate for a recent college graduate
    - no experience needed and requires advanced education
- UK based
    - "graduate" means college graduate
    

The metric used is "precision at 95% recall"
- given that the model achieves a recall of at least 95%
$$
\frac{\text{TP}}{\text{TP} + \text{FN}} \ge 95%
$$
- what is the precision (predicted Positives that are True Positives;minimize FP)
$$
\frac{\text{TP}}{\text{TP} + \text{FP}}
$$

The models evaluated are two variants of GPT 3.5 via OpenAI.

## Prompt modifications evaluated


<table>
    <center><strong>Prompt modifications</strong></center>
    <tr>
        <img src="images/prompt_eng_case_study.png" width=80%>
    </tr>
    
    Attribution: https://arxiv.org/pdf/2303.07142.pdf#page=7
</table>

### Baseline

Uses Keyword and Regular Expression search
- "Graduate" or "Junior" in job title
- "suitable for graduate" in body of posting


### Chain of Thought (CoT): Few Shot

Provide one or more exemplars for the task
- where the exemplar demonstrates the correct response

    
The exemplars condition the LLM to produces responses
- that look like the exemplars
- so if the exemplars demonstrate step by step reasoning
- the responses will hopefully do the same

See panel (b) in the chart below
 

<table>
    <center><strong>Chain of Thought Prompting</strong></center>
    <tr>
        <img src="images/cot_step_by_step.png" width=80%>
    </tr>
    
    Attribution: https://arxiv.org/pdf/2201.11903.pdf
</table>

### Zero CoT: Chain of Thought: Zero shot

- Append "Let's think step by step" to the base prompt
    - see panel (d) in the chart above

### Instructions: variants

The prompt uses [Role prompting](https://learnprompting.org/docs/basics/roles)
- the *role* the Assistant is to play in providing the response

>You are an AI expert in career advice. You are tasked
with sorting through jobs by analysing their content and
deciding whether they would be a good fit for a recent
graduate or not.
    
- giving [instructions](https://learnprompting.org/docs/basics/instructions) describing the task

>A job is fit for a graduate if it's a junior-level
position that does not require extensive prior professional
experience. I will give you a job posting and you will
analyse it, to know whether or not it describes a position
fit for a graduate.

The experiment was carried out in the [OpenAI Playground](https://platform.openai.com/playground).

This tool has multiple input fields (System, User)
- the following prompt techniques refer to placement in specific input areas
- the actual prompt concatenates the two: System + User

<img src="images/openai_playground.png">

#### `rawinst`

Role and Instruction placed in User Query field (top middle of page)

#### `sysint`

Role and Instruction placed in System Query field (to left of page)

#### `bothinst`

Role placed in System Query field; Instruction placed in User Query field

### Mocked exchange (`mock`)

This
- creates an initial prompt to the Assistant, requesting that it confirm it's understanding ("Got it ?") of the Instructions in the User Query field
- the prompt and response are then used as the value of the User Query field

Variant of `bothinst` where 
- the User field becomes
    >A job is fit for a graduate ... Got it ?
    
    >[Assistant response] Yes, I understand. I am ready to analyse your job posting.


### Reiterating instructions (`reit`)

Both the Role and the Instruction are reinforced by repetition.

- Role
    >You are an AI expert in career advice.  ... 

    >**Remember, you're the best AI careers expert
    and will use your expertise to provide the best possible
    analysis**
- Instruction
    >A job is fit for a graduate ...
    >and you will analyse it, **step-by-step,** to know whether or not it describes ...


### Wording the prompt

These involve appending the desired format of the response to the Instruction

####  `loose`
>Your answer must end with:

>Final Answer: This is a (A) job fit for a recent graduate or
a student OR (B) a job requiring more professional experience.

>Answer: Let's think step-by-step,


#### `strict`

>You will answer following this template:

>Reasoning step 1:

>Reasoning step 2:

>Reasoning step 3

>Final Answer: This is a (A) job fit for a recent graduate or
a student OR (B) a job requiring more professional experience.

>Answer: Reasoning Step 1:

### Right conclusion (`right`)

Ask the model for step-by-step reasoning in order to arrive at the **right conclusion**

> Let's think step-by-step **to reach the right conclusion,**

Again, let's remember that the LLM is performing text completion
- one token at a time

If any token in the sequence of results is not great
- the final result will also not be great.

The "right conclusion" prompt may cause the LLM to 
- re-evaluate how good an intermediate result is
- rather than solely focusing on the high probability next token objective

### Reasoning gaps (`info`)

Prevent mis-interpretation of the Instructions

>A job is fit for a graduate if it's a junior-level
position that does not require extensive prior professional
experience. 

>**When analysing the experience required, take
into account that requiring internships is still fit for a
graduate.**

>I will give you a job posting and you will
analyse it, ...

### Subtle tweaks

#### Giving the assistant a name (`name`)

Give the assistant a name when describing its role.

Change the Role from
>You are an AI expert in career advice ...

to
>You are Sydney, an AI expert in career advice ...

#### Positive feedback (`pos`)

A modification of Mocked Exchange
- after the Assistant confirms its understanding
- give it positive feedback in the form of
>Great! Let's begin then :)

before continuing the mocked exchange

## Evaluation

The results are summarized in a table
- Sub-metrics are reported to facilitate comparison
    - rather than the ultimate metric of "precision at 95% recall"
- *Template stickiness* refers to the format of the response
    - the fraction of responses that conform to the desired format
    - and don't need further parsing
        - important for downstream uses and ease of evaluation
    

<table>
    <center><strong>Evaluation of Prompt modifications</strong></center>
    <tr>
        <img src="images/prompt_eng_case_study_results.png" width=80%>
    </tr>
    
    Attribution: https://arxiv.org/pdf/2303.07142.pdf#page=12
</table>

### High level conclusions
- Prompt formatting is important
    - Final prompt greatly improves over Baseline
        - F1: $65.6 \leadsto 91.7$
        - Recall: $70.6 \leadsto 97$

- Chain of Thought: more exemplars **don't improve** performance
    - Zero shot(`Zero-CoT`) vs Few shot (`CoT`)
        - F1: $81.4 \leadsto  78.4$
        - Precision $75.5 \leadsto  72.6$
    - Theories
        - the task was sufficiently simple that exemplars weren't needed
        - the exemplars thus
            - increased Recall
            - but decrease Precision
        - We mentioned another theory in the [In Context Learning theory module](Prompt_Engineering_Suggestions.ipynb#Prompt-Programming-for-Large-Language-Models:-Beyond-the-Few-Shot-Paradigm)
            - the role of exemplars is to *help locate* the desired task among the tasks seen in training

- Role Prompting and Instructions lead to the biggest increase in performance under Zero shot CoT
    - `Zero-CoT` $\leadsto$ `+rawinst`
        - F1: $81.4 \leadsto 85.8$
        - Precision: $75.5 \leadsto 80$
- Placement of Role and Instruction with Query fields is significant
    - Why ?  Must not be simple concatenation as I conjectured
    - `Zero-CoT` $\leadsto$ `bothinst`
        - F1: $81.4 \leadsto 87.5$
        - Precision: $75.5 \leadsto 81.9$

- Mocked exchange increase Recall
    - `bothinst` $\leadsto$ `bothinst + mock`
        - Recall: $93.9 \leadsto 95.1$
        - above the desired 95% Recall threshold for the ultimate metric "precision at 95% recall"

- Repetition in prompts helps !  Combining leads to Final result that dominates all others.
    - Re-iterating instructions (`reit`)
    - Emphasizing Right conclusion (`right`)
    - Emphasizing elminating Reasoning gaps ('info`)

In [2]:
print("Done")

Done
